COMPSCI 723: NLP

Ukraine Russia War Twitter Sentiment Analysis

-Shreyas Chate

-Mihir Dixit

datasetlink > https://drive.google.com/file/d/1x13lLQ8mhp5jIYN8sCpt4qgGBl4whf6i/view?usp=sharing

In [ ]:
import numpy as np 
import pandas as pd 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import csv
import warnings
import os
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import nltk
from nltk.corpus import stopwords
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models
from sklearn.feature_extraction.text import CountVectorizer
from collections import  Counter

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

warnings.simplefilter(action='ignore')


In [ ]:
file1 = r"./kaggle/input/UkraineCombinedTweetsDeduped_FEB.csv.gzip"
file2  = r"./kaggle/input/UkraineCombinedTweetsDeduped_MAR.csv.gzip"

dataframe1 = pd.read_csv(file1, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
dataframe2 = pd.read_csv(file2, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)


dataframef = pd.concat([dataframe1])
dataframe = pd.concat([dataframef,dataframe2])

tweet = dataframe[dataframe['language'] == 'en'] 
tweet_dataframe = tweet.loc[:,['text']]  
tweet_dataframe.head()
len(tweet_dataframe)

c=len(tweet_dataframe)//8
tweets_dataframe=tweet_dataframe.iloc[:c]
len(tweets_dataframe)


In [ ]:
def CleanTweet(text):

    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
    text = re.sub(r'\w*\d\w*', '', text)  
    text = re.sub(r'http\S+', '', text)
   
    emoji_patterns = re.compile(pattern = "["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF" 
                           u"\U0001F680-\U0001F6FF" 
                           u"\U0001F1E0-\U0001F1FF" 
                           u"\U00002702-\U000027B0"  
                           u"\U000024C2-\U0001F251" 
                           "]+", flags = re.UNICODE)
    text = emoji_patterns.sub(r'', text)
    return text

tweetsdataframe = pd.DataFrame(tweets_dataframe.text.apply(lambda x: CleanTweet(x)))
tweetsdataframe

plt.figure(figsize=(10,7))
txt_length = [len(d) for d in tweetsdataframe.text]
plt.hist(txt_length, bins=100)
plt.title('Distribution of Tweets')
plt.ylabel('Number of Tweets')
plt.xlabel('Tweets length')
sns.despine()


In [ ]:

stopwords_set = set(STOPWORDS)
wordcloud = WordCloud(background_color='white',
                     stopwords = stopwords_set,
                      max_words = 300,
                      max_font_size = 40,
                      random_state=42).generate(str(tweetsdataframe['text']))

print(wordcloud)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()



In [ ]:
dataframe_en=dataframe[dataframe["language"] == "en"].location.value_counts().sort_values(ascending=False)[:100]
dataframe_en= pd.DataFrame(dataframe_en)
dataframe_en=dataframe_en.reset_index()

longitude,latitude=[],[]
GeoLocation = Nominatim(user_agent="MyApp")
for i in range(len(dataframe_en)):
    location = GeoLocation.geocode(dataframe_en['index'][i])
    longitude.append(location.longitude)
    latitude.append(location.latitude)

dataframe_en.insert(1, "latitude",latitude, True)
dataframe_en.insert(2, "longitude",longitude, True)

LocationMap= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(LocationMap)

for i in range(len(dataframe_en)):
        latitude = dataframe_en.iloc[i]['latitude']
        longitude = dataframe_en.iloc[i]['longitude']
        radius=5
        folium.CircleMarker(location = [latitude, longitude], radius=radius, fill =True).add_to(marker_cluster)
LocationMap


In [ ]:
def PlotBarChart(text, n=2):
    stop=set(stopwords.words('english'))

    new= text.str.split()
    new=new.values.tolist()
    corpus=[word for i in new for word in i]

    def nGrams(corpus, n=None):
        vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:10]

    top_n_bigrams=nGrams(text,n)[:10]
    x,y=map(list,zip(*top_n_bigrams))
    sns.barplot(x=y,y=x)


In [ ]:
from matplotlib.font_manager import mpl
mpl.rcParams['figure.figsize']=(8.0,8.0)  
PlotBarChart(tweetsdataframe['text'],4)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')


stop_words = stopwords.words('english')
stop_words += list(string.punctuation) 
stop_words += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
def TextTokenizer(text):
    tokens = nltk.word_tokenize(text)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stop_words]
    return stopwords_removed



In [ ]:
!pip install pyLDAvis
from gensim.corpora.dictionary import Dictionary
from pyparsing.helpers import Dict
tweetsdataframe['text']=tweetsdataframe['text'].apply(TextTokenizer)
TexttoDict = Dictionary(tweetsdataframe['text'])
TexttoDict.token2id
TweetsX = [TexttoDict.doc2bow(tweet) for tweet in tweetsdataframe['text']]
TweetsX

LdaTweets = LdaModel(TweetsX,
                      num_topics = 10,
                      id2word = TexttoDict,
                      random_state = 1,
                      passes=10)

LdaTweets.show_topics()

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
LDAvis = gensimvis.prepare(LdaTweets, TweetsX, dictionary=LdaTweets.id2word)
LDAvis


In [ ]:
!pip install flair
from flair.models import TextClassifier
from flair.data import Sentence
loadClassifier = TextClassifier.load('en-sentiment')


In [ ]:
!pip install swifter

tweets_dataframe['text'] = pd.DataFrame(tweets_dataframe.text.apply(lambda x: clean_text(x)))
tweets_dataframe=tweets_dataframe.sample(frac=0.33,random_state=1)

def SentimentPrediction(x):

    newTweet = Sentence(x)
    
    try:        
        loadClassifier.predict(newTweet)
        score = newTweet.labels[0]
        staging_score = str(score).replace("(",",").replace(")","")
        
        sentiment_score = staging_score.split(",")
        
        if "POSITIVE" in str(sentiment_score[0]):
            return "POSITIVE", float(sentiment_score[1].strip())
        elif "NEGATIVE" in str(sentiment_score[0]):
            return "NEGATIVE", float(sentiment_score[1].strip())
        else:
            return "NEUTRAL", 0.00
    except Exception:
        print(newTweet)
        pass  
    
    return "ERROR",0.00

tweets_dataframe['Sentiment'] = ""
tweets_dataframe['Sentiment_Score'] = np.nan


In [ ]:

import swifter
tweets_dataframe["Sentiment"],tweets_dataframe["Sentiment_Score"] =  zip(*tweets_dataframe["text"].swifter.apply(SentimentPrediction))
tweets_dataframe